In [ ]:
import subprocess
import sys
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"    # Hide most TF/XLA logs
os.environ['XLA_FLAGS'] = '--xla_cpu_multi_thread_eigen=false intra_op_parallelism_threads=1'
os.environ["BITSANDBYTES_NOWELCOME"] = "1"  # Suppress warning
os.environ["LOAD_IN_4BIT_FORCE"] = "1"      # Force Unsloth to try loading in 4-bit


def install_dependencies():
    """Install required packages for Kaggle"""
    packages = [
        'torch>=2.0.0',
        'opencv-python',
        'pillow',
        'librosa',
        'matplotlib',
        'numpy',
        'pandas',
        'onnx',
        'onnxruntime',
        'bitsandbytes',
        'transformers>=4.36.0',
        'accelerate',
        'sentencepiece',
        'protobuf==3.20.*',
        "trl"
    ]
    
    print("🔄 Installing dependencies...")
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-q","--no-deps", package])
            print(f"✅ {package} installed")
        except subprocess.CalledProcessError:
            print(f"⚠️ Failed to install {package}")


# Try to install Unsloth for Kaggle
import subprocess, sys, os

try:
     if "KAGGLE_KERNEL_RUN_TYPE" in os.environ:
        print("🔄 Installing Unsloth and Unsloth Zoo (no deps)...")

        # Install unsloth
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", "-q", "--no-deps",
            "git+https://github.com/unslothai/unsloth.git"
        ])

        # Install unsloth_zoo
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", "-q", "--no-deps",
            "git+https://github.com/unslothai/unsloth_zoo.git"
        ])
     else:
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", "-q", "--no-deps", "unsloth"
        ])
        subprocess.check_call([
            sys.executable, "-m", "pip", "install", "-q", "--no-deps", "unsloth_zoo"
        ])
    
     print("✅ Unsloth and Zoo installed successfully")
except Exception as e:
     print("⚠️ Installation failed - using fallback")
     print(e)

# Install dependencies
install_dependencies()


# Core imports
import torch
import numpy as np
import pandas as pd
from PIL import Image
import librosa
import cv2
import json
import matplotlib.pyplot as plt
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import logging
import gradio as gr

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
print("✅ Imported all core modules successfully!")

# Check for Unsloth
try:
    from unsloth import FastLanguageModel
    UNSLOTH_AVAILABLE = True
    print("✅ Unsloth available")
except ImportError:
    UNSLOTH_AVAILABLE = False
    print("⚠️ Using standard transformers")

# Check for ONNX
try:
    import onnx
    import onnxruntime as ort
    ONNX_AVAILABLE = True
    print("✅ ONNX available")
except ImportError:
    ONNX_AVAILABLE = False
    print("⚠️ ONNX not available")



class PlantHealthAdvisor:
    """Clean, focused plant health analysis system for Gradio - UNSLOTH ONLY"""
    
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model = None
        self.tokenizer = None
        self.audio_processor = None
        
        # Create output directory
        self.output_dir = Path("./plant_analysis_output")
        self.output_dir.mkdir(exist_ok=True)
        
        # Plant health knowledge base
        self.health_conditions = {
            'healthy': {
                'symptoms': ['green leaves', 'upright growth', 'no discoloration'],
                'treatment': 'Continue current care. Monitor regularly for changes.'
            },
            'fungal_disease': {
                'symptoms': ['white spots', 'black spots', 'yellowing', 'powdery coating'],
                'treatment': 'Apply fungicide spray. Improve air circulation. Reduce watering frequency.'
            },
            'nutrient_deficiency': {
                'symptoms': ['yellow leaves', 'stunted growth', 'pale color', 'brown edges'],
                'treatment': 'Apply balanced fertilizer. Check soil pH. Ensure proper nutrients.'
            },
            'pest_damage': {
                'symptoms': ['holes in leaves', 'chewed edges', 'visible insects'],
                'treatment': 'Use organic pesticide. Remove affected parts. Check for pest eggs.'
            },
            'water_stress': {
                'symptoms': ['wilting', 'drooping', 'dry soil', 'brown tips'],
                'treatment': 'Adjust watering schedule. Check soil moisture. Improve drainage.'
            }
        }
        
        print("🌿 PlantHealthAdvisor initialized (Unsloth-only mode)")
    
    def load_model(self):
        """Load Unsloth model - NO FALLBACK to transformers"""
        try:
            return self._load_unsloth_model()
        except Exception as e:
            logger.error(f"Unsloth model loading failed: {e}")
            raise RuntimeError(f"Failed to load Unsloth model: {e}. No fallback available.")
    

    def _load_unsloth_model(self):
        """Load Unsloth model for better performance"""
        try:
            model_name = "unsloth/gemma-2-2b-it-bnb-4bit"  # Smaller model for Kaggle
            print(f"🔄 Loading Unsloth model: {model_name}")
            
            self.model, self.tokenizer = FastLanguageModel.from_pretrained(
                model_name=model_name,
                max_seq_length=1024,
                load_in_4bit=True,
            )
            
            FastLanguageModel.for_inference(self.model)
            print("✅ Unsloth model loaded successfully")
            return True
            
        except Exception as e:
            print(f"❌ Unsloth model failed: {e}")
            return self._load_standard_model()
    
    def load_audio_processor(self):
        """Load audio processing pipeline"""
        try:
            self.audio_processor = pipeline(
                "automatic-speech-recognition",
                model="openai/whisper-tiny.en",
                device=0 if torch.cuda.is_available() else -1
            )
            print("✅ Audio processor loaded")
            return True
        except Exception as e:
            print(f"⚠️ Audio processor failed: {e}")
            return False
    
    def process_image(self, image):
        """Extract features from plant image"""
        try:
            # Handle PIL Image directly
            if isinstance(image, np.ndarray):
                image = Image.fromarray(image)
            
            # Ensure RGB format
            if image.mode != 'RGB':
                image = image.convert('RGB')
                
            image_resized = image.resize((224, 224))
            image_array = np.array(image_resized)
            
            # Extract basic visual features
            features = {
                'brightness': float(np.mean(image_array)),
                'contrast': float(np.std(image_array)),
                'green_dominance': float(np.mean(image_array[:,:,1]) / np.mean(image_array)),
                'color_variance': float(np.var(image_array.reshape(-1, 3), axis=0).mean()),
                'image_shape': image.size
            }
            
            return features, image
            
        except Exception as e:
            logger.error(f"Image processing failed: {e}")
            return None, None
    
    def process_audio(self, audio_input):
        """Convert audio to text description"""
        try:
            if self.audio_processor is None:
                if not self.load_audio_processor():
                    return "Audio processing not available"
            
            # Handle different audio input types
            if audio_input is None:
                return ""
            
            # If it's a tuple (sample_rate, audio_array) from Gradio microphone
            if isinstance(audio_input, tuple):
                sample_rate, audio_data = audio_input
                # Save temporary audio file
                with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as tmp_file:
                    import soundfile as sf
                    sf.write(tmp_file.name, audio_data, sample_rate)
                    audio_path = tmp_file.name
            else:
                # It's a file path
                audio_path = audio_input
            
            # Process audio file
            result = self.audio_processor(audio_path)
            text = result.get('text', '')
            
            # Clean up temporary file if created
            if isinstance(audio_input, tuple) and os.path.exists(audio_path):
                os.unlink(audio_path)
            
            print(f"🎤 Audio transcribed: {text}")
            return text
            
        except Exception as e:
            logger.error(f"Audio processing failed: {e}")
            return f"Audio processing error: {str(e)}"
    
    def analyze_plant_health(self, image, audio_input=None, text_description=""):
        """Main analysis function for Gradio"""
        try:
            print("🌿 Starting plant health analysis...")
            
            # Process image
            image_features, processed_image = self.process_image(image)
            if image_features is None:
                return "Error: Failed to process image", None, {}
            
            # Process audio if provided
            audio_text = ""
            if audio_input is not None:
                audio_text = self.process_audio(audio_input)
            
            # Combine descriptions
            full_description = f"{text_description} {audio_text}".strip()
            
            # Generate AI analysis - UNSLOTH REQUIRED
            if self.model is None:
                raise RuntimeError("Unsloth model not loaded. Cannot perform AI analysis.")
            
            analysis = self._generate_unsloth_analysis(image_features, full_description)
            
            # Get recommendations
            recommendations = self._get_recommendations(analysis, image_features)
            
            # Calculate confidence
            confidence = self._calculate_confidence(analysis, image_features)
            
            # Create result
            result = {
                'timestamp': datetime.now().isoformat(),
                'image_features': image_features,
                'description': full_description,
                'analysis': analysis,
                'recommendations': recommendations,
                'confidence': confidence
            }
            
            # Create formatted output
            output_text = self._format_output(result)
            
            # Create visualization
            viz_image = self._create_visualization_image(processed_image, result)
            
            print("✅ Analysis complete!")
            return output_text, viz_image, result
            
        except Exception as e:
            error_msg = f"Analysis failed: {str(e)}"
            logger.error(error_msg)
            return error_msg, None, {}
    
    def _generate_unsloth_analysis(self, image_features, description):
        """Generate AI analysis using UNSLOTH ONLY"""
        try:
            # Create analysis prompt
            prompt = f"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert plant health specialist. Analyze the provided plant data and give a detailed health assessment.

<|start_header_id|>user<|end_header_id|>
Plant Health Analysis:

Image Analysis:
- Brightness: {image_features['brightness']:.1f}
- Contrast: {image_features['contrast']:.1f}  
- Green dominance: {image_features['green_dominance']:.2f}
- Color variance: {image_features['color_variance']:.1f}

Description: {description if description else 'No description provided'}

Based on this information, analyze the plant's health status, identify any issues, and suggest the most likely condition. Be concise and specific.

<|start_header_id|>assistant<|end_header_id|>
Analysis:"""
            
            # Generate response using Unsloth
            inputs = self.tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512)
            
            if torch.cuda.is_available():
                inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=200,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.tokenizer.eos_token_id,
                    eos_token_id=self.tokenizer.eos_token_id,
                )
            
            # Decode response
            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            analysis = response.replace(prompt, "").strip()
            
            return analysis if analysis else "Unsloth model generated empty response"
            
        except Exception as e:
            logger.error(f"Unsloth analysis failed: {e}")
            raise RuntimeError(f"Unsloth AI analysis failed: {e}")
    
    def _get_recommendations(self, analysis, image_features):
        """Generate specific recommendations"""
        recommendations = []
        analysis_lower = analysis.lower()
        
        # Match analysis with known conditions
        for condition, info in self.health_conditions.items():
            if condition.replace('_', ' ') in analysis_lower:
                recommendations.append({
                    'condition': condition,
                    'treatment': info['treatment'],
                    'confidence': 0.8
                })
        
        # Add general recommendations based on image features
        if image_features['brightness'] < 80:
            recommendations.append({
                'condition': 'lighting',
                'treatment': 'Consider providing more light or moving to brighter location',
                'confidence': 0.6
            })
        
        if image_features['green_dominance'] < 0.85:
            recommendations.append({
                'condition': 'general_health',
                'treatment': 'Monitor plant closely and check for signs of stress or disease',
                'confidence': 0.7
            })
        
        # Default recommendation if none found
        if not recommendations:
            recommendations.append({
                'condition': 'maintenance',
                'treatment': 'Continue regular care routine. Monitor for any changes in plant health.',
                'confidence': 0.5
            })
        
        return recommendations
    
    def _calculate_confidence(self, analysis, image_features):
        """Calculate confidence score for the analysis"""
        confidence = 0.5  # Base confidence
        
        # Increase confidence based on image quality
        if image_features['brightness'] > 80:
            confidence += 0.1
        if image_features['contrast'] > 40:
            confidence += 0.1
        
        # Increase confidence if specific conditions are identified
        if any(condition in analysis.lower() for condition in self.health_conditions.keys()):
            confidence += 0.2
        
        return min(confidence, 0.95)  # Cap at 95%
    
    def _format_output(self, result):
        """Format analysis results for display"""
        output = f"""
🌿 **PLANT HEALTH ANALYSIS RESULTS** (Powered by Unsloth)
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🎯 **Confidence Score:** {result['confidence']:.1%}

📊 **Image Analysis:**
• Brightness: {result['image_features']['brightness']:.1f}
• Contrast: {result['image_features']['contrast']:.1f}
• Green Dominance: {result['image_features']['green_dominance']:.2f}
• Color Variance: {result['image_features']['color_variance']:.1f}

💬 **Description:** {result['description'] or 'No description provided'}

🔍 **AI Analysis (Unsloth):**
{result['analysis']}

💡 **Recommendations:**
"""
        
        for i, rec in enumerate(result['recommendations'], 1):
            output += f"{i}. **{rec['condition'].replace('_', ' ').title()}**: {rec['treatment']} (Confidence: {rec['confidence']:.1%})\n"
        
        output += f"\n⏰ **Analysis Time:** {result['timestamp']}"
        
        return output
    
    def _create_visualization_image(self, image, result):
        """Create analysis visualization as image"""
        try:
            fig, axes = plt.subplots(1, 2, figsize=(15, 6))
            
            # Display image
            axes[0].imshow(image)
            axes[0].set_title('Plant Image', fontsize=14, fontweight='bold')
            axes[0].axis('off')
            
            # Display analysis results
            analysis_text = f"""PLANT HEALTH ANALYSIS (Unsloth)
            
Confidence: {result['confidence']:.1%}

Image Features:
• Brightness: {result['image_features']['brightness']:.1f}
• Contrast: {result['image_features']['contrast']:.1f}
• Green Dominance: {result['image_features']['green_dominance']:.2f}

Analysis:
{result['analysis'][:150]}...

Top Recommendations:"""
            
            for i, rec in enumerate(result['recommendations'][:2], 1):
                analysis_text += f"\n{i}. {rec['treatment'][:40]}..."
            
            axes[1].text(0.05, 0.95, analysis_text, transform=axes[1].transAxes,
                        fontsize=10, verticalalignment='top', fontfamily='monospace',
                        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8))
            axes[1].set_title('Analysis Results (Unsloth)', fontsize=14, fontweight='bold')
            axes[1].axis('off')
            
            plt.tight_layout()
            
            # Convert to image
            buf = io.BytesIO()
            plt.savefig(buf, format='png', dpi=150, bbox_inches='tight')
            buf.seek(0)
            viz_image = Image.open(buf)
            plt.close()
            
            return viz_image
            
        except Exception as e:
            logger.error(f"Visualization failed: {e}")
            return None

# Initialize the advisor
advisor = PlantHealthAdvisor()

def analyze_plant(image, audio, text_description, load_model_first):
    """Main function for Gradio interface - UNSLOTH REQUIRED"""
    global advisor
    
    if image is None:
        return "Please provide an image of the plant.", None
    
    # ALWAYS load Unsloth model - no choice
    if advisor.model is None:
        try:
            print("🚀 Loading Unsloth model (required)...")
            status = advisor.load_model()
            if not status:
                return "❌ CRITICAL ERROR: Unsloth model failed to load. Cannot proceed.", None
        except Exception as e:
            return f"❌ CRITICAL ERROR: Unsloth model loading failed: {e}", None
    
    # Run analysis
    try:
        result_text, viz_image, result_data = advisor.analyze_plant_health(
            image, audio, text_description
        )
        return result_text, viz_image
    except Exception as e:
        return f"❌ Analysis failed: {e}", None

def create_demo_image(condition):
    """Create demo images for testing"""
    try:
        img = Image.new('RGB', (400, 500), color='lightblue')
        draw = ImageDraw.Draw(img)
        
        # Draw pot
        draw.ellipse([100, 400, 300, 470], fill='brown')
        
        # Draw stem
        draw.rectangle([195, 250, 205, 400], fill='darkgreen')
        
        # Draw leaves based on condition
        leaf_color = 'green'
        if condition == 'sick':
            leaf_color = 'yellow'
        elif condition == 'pest_damage':
            leaf_color = 'darkgreen'
        
        # Draw multiple leaves
        import random
        for i in range(8):
            x = 200 + random.randint(-80, 80)
            y = 120 + i * 30
            draw.ellipse([x-30, y-15, x+30, y+15], fill=leaf_color)
            
            # Add damage for sick plants
            if condition == 'sick':
                draw.ellipse([x-8, y-5, x+8, y+5], fill='brown')
            elif condition == 'pest_damage':
                draw.ellipse([x-5, y-3, x+5, y+3], fill='black')
        
        return img
    except Exception as e:
        print(f"Demo image creation failed: {e}")
        return None

# Create Gradio interface
with gr.Blocks(title="🌿 CropSenseAI - Plant Health Advisor (Unsloth Only)", theme=gr.themes.Soft()) as app:
    gr.Markdown("""
    # 🌿 CropSenseAI - AI Plant Health Advisor (Unsloth Powered)
    
    Upload an image of your plant and get AI-powered health analysis and treatment recommendations!
    
    **Features:**
    - 📸 Image analysis with computer vision
    - 🎤 Voice description support
    - 🚀 **Unsloth-powered AI diagnosis (REQUIRED)**
    - 💡 Treatment recommendations
    - 📊 Confidence scoring
    
    ⚠️ **Note: This application requires Unsloth to function. No fallback available.**
    """)
    
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 📸 Plant Image")
            image_input = gr.Image(
                label="Upload plant image or use camera",
                type="pil",
                sources=["upload", "webcam"]
            )
            
            gr.Markdown("### 🎤 Audio Description (Optional)")
            audio_input = gr.Audio(
                label="Record or upload audio description",
                type="filepath",
                sources=["microphone", "upload"]
            )
            
            gr.Markdown("### 📝 Text Description (Optional)")
            text_input = gr.Textbox(
                label="Describe plant condition",
                placeholder="e.g., 'Plant has yellow spots on leaves and looks droopy'",
                lines=3
            )
            
            gr.Markdown("⚠️ **Unsloth model will be loaded automatically (required)**")
            
            with gr.Row():
                analyze_btn = gr.Button("🔍 Analyze Plant Health (Unsloth)", variant="primary", size="lg")
                clear_btn = gr.Button("🗑️ Clear", variant="secondary")
        
        with gr.Column(scale=1):
            gr.Markdown("### 📊 Analysis Results")
            result_output = gr.Markdown(label="Analysis Results")
            
            gr.Markdown("### 📈 Visualization")
            viz_output = gr.Image(label="Analysis Visualization")
    
    # Demo section
    with gr.Row():
        gr.Markdown("### 🎬 Try Demo Images")
        
        with gr.Row():
            demo_healthy_btn = gr.Button("🌱 Healthy Plant Demo", variant="secondary")
            demo_sick_btn = gr.Button("🤒 Sick Plant Demo", variant="secondary")
            demo_pest_btn = gr.Button("🐛 Pest Damage Demo", variant="secondary")
    
    # Event handlers
    analyze_btn.click(
        fn=analyze_plant,
        inputs=[image_input, audio_input, text_input, gr.State(True)],  # Always load model
        outputs=[result_output, viz_output]
    )
    
    clear_btn.click(
        fn=lambda: (None, None, "", "", None),
        outputs=[image_input, audio_input, text_input, result_output, viz_output]
    )
    
    # Demo button handlers
    demo_healthy_btn.click(
        fn=lambda: create_demo_image("healthy"),
        outputs=image_input
    )
    
    demo_sick_btn.click(
        fn=lambda: create_demo_image("sick"),
        outputs=image_input
    )
    
    demo_pest_btn.click(
        fn=lambda: create_demo_image("pest_damage"),
        outputs=image_input
    )
    
    # System info
    gr.Markdown(f"""
    ### ℹ️ System Information
    - 🐍 Python: {sys.version_info.major}.{sys.version_info.minor}
    - 🔥 PyTorch: Available
    - 💾 CUDA: {torch.cuda.is_available()}
    - 🚀 **Unsloth: REQUIRED (No alternatives)**
    """)

In [ ]:
# Launch the app
if __name__ == "__main__":
    print("🌿 Starting CropSenseAI Gradio Interface (Unsloth Only)...")
    print("🚀 Unsloth is REQUIRED for this application to function...")
    
    # Pre-load audio processor for faster first-time use
    try:
        advisor.load_audio_processor()
    except:
        print("⚠️ Audio processor pre-loading failed")
    
    app.launch(
        share=True,
        server_name="0.0.0.0",
        server_port=7860,
        show_error=True
    )